In this notebook we will learn how to create and train an embedding layer for the words appearing in a text data. We will then train a simple DNN based model to do sentiment analysis on this data. 

## Exercise

This is exercise 13.10 in [this](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) book.

### Problem Statement

In this exercise you will download a dataset, split it, create a tf.data.Dataset to load it and preprocess it efficiently, then build and train a binary classification model containing an Embedding layer:

  - a. Download the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/), which contains 50,000 movies reviews from the Internet Movie Database. The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders (including preprocessed bag-of-words), but we will ignore them in this exercise.
  
    
  - b. Split the test set into a validation set (15,000) and a test set (10,000).
  
  
  - c. Use tf.data to create an efficient dataset for each set.
  
  
  - d. Create a binary classification model, using a TextVectorization layer to preprocess each review. If the TextVectorization layer is not yet available (or if you like a challenge), try to create your own custom preprocessing layer: you can use the functions in the tf.strings package, for example lower() to make everything lowercase, regex_replace() to replace punctuation with spaces, and split() to split words on spaces. You should use a lookup table to output word indices, which must be prepared in the adapt() method.
  
  
  - e. Add an Embedding layer and compute the mean embedding for each review, multiplied by the square root of the number of words (see Chapter 16). This rescaled mean embedding can then be passed to the rest of your model.
  
  
  - f. Train the model and see what accuracy you get. Try to optimize your pipelines to make training as fast as possible.


  - g. Use TFDS to load the same dataset more easily: tfds.load("imdb_reviews").

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print('tensorflow version: {}'.format(tf.__version__))
print('keras version: {}'.format(keras.__version__))

tensorflow version: 2.1.0
keras version: 2.2.4-tf


In [2]:
import os
print('cwd: {}'.format(os.getcwd()))

cwd: /home/prarit/MachineLearningProjects/Word-Embeddings


### Downloading the Large Movie Review Dataset

In [3]:
# good tutorial on using wget: https://www.tecmint.com/download-and-extract-tar-files-with-one-command/
# turn off verbose output of wget using the flag -nv : https://shapeshed.com/unix-wget/#how-to-turn-off-verbose-output 
!wget -c -nv http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -o - 

In [4]:
# uncompress the downloaded files
!tar xzf aclImdb_v1.tar.gz

Note that tensorflow also provides this dataset: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb

### Briefly Explore the Dataset

In [5]:
# list the files in the current working directory
os.listdir()

['README.md',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'aclImdb',
 'Word-Embeddings.ipynb',
 'aclImdb_v1.tar.gz']

We see that aclImdb_v1.tar.gz was extracted to a folder called aclImdb. Let's see the contents of this file.

In [6]:
path = os.path.join(os.getcwd() , 'aclImdb')
contents = os.listdir(path)
print('The contents of aclImdb are: \n{}'.format(contents))

The contents of aclImdb are: 
['imdb.vocab', 'train', 'README', 'imdbEr.txt', 'test']


There is a README file in aclImdb, let us read it.

In [7]:
# read README
filepath = os.path.join(path, 'README')
with open(filepath, 'r') as f:
    print(f.read())

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

From README, we find that the train/test folder contains a 'pos' folder for positive reviews and a 'neg' for negative reviews along with .txt files containing urls of positive and negative reviews respectively. There are also some other files for bag-of-words features etc. 

Each train/test folder contains a total of 25000 reviews of which 12500 are positive reviews and 12500 are negative reviews.

Let's verify the above about the 'train' folder

In [8]:
train_path = os.path.join(path,'train')
print('contents of the train folder: \n{}'.format(os.listdir(train_path)))

contents of the train folder: 
['unsup', 'urls_pos.txt', 'pos', 'labeledBow.feat', 'unsupBow.feat', 'neg', 'urls_neg.txt', 'urls_unsup.txt']


In order to create a dataset, we need the path to all the reviews. We can create the corresponding list of paths using glob

In [9]:
import glob

In [10]:
# paths to the positive reviews in the training set
train_pos_path = os.path.join(train_path, 'pos', '*.txt')
train_pos_reviews = glob.glob(train_pos_path)
print('No. of train-set files with positive reviews: {}'.format(len(train_pos_reviews)))

No. of train-set files with positive reviews: 12500


In [11]:
# paths to the negative reviews in the training set
train_neg_path = os.path.join(train_path, 'neg', '*.txt')
train_neg_reviews = glob.glob(train_neg_path)
print('No. of train-set files with negative reviews: {}'.format(len(train_neg_reviews)))

No. of train-set files with negative reviews: 12500


let us give a brief look at a positive review. 

In [12]:
file = train_pos_reviews[0]
with open(file, 'r') as f:
    print(f.read())

I watch them all.<br /><br />It's not better than the amazing ones (_Strictly Ballroom_, _Shall we dance?_ (Japanese version), but it's completely respectable and pleasingly different in parts.<br /><br />I am an English teacher and I find some of the ignorance about language in some of these reviews rather upsetting. For example: the "name should scream don't watch. 'How she move.' Since when can movie titles ignore grammar?" <br /><br />There is nothing inherently incorrect about Caribbean English grammar. It's just not Canadian standard English grammar. Comments about the dialogue seem off to me. I put on the subtitles because I'm a Canadian standard English speaker, so I just AUTOMATICALLY assumed that I would have trouble understanding all of it. It wasn't all that difficult and it gave a distinctly different flavour as the other step movies I have seen were so American.<br /><br />I loved that this movie was set in Toronto and, in fact, wish it was even more clearly set there. I 

Later, we will like to load an preprocess all the data using tensorflow's data API, therefore let us quickly see how to read the same file as above but this time by using tensorflow's [TextLineDataset](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) method.

In [13]:
pos_fl0 = tf.data.TextLineDataset(file)
for item in pos_fl0:
    print(item)

tf.Tensor(b'I watch them all.<br /><br />It\'s not better than the amazing ones (_Strictly Ballroom_, _Shall we dance?_ (Japanese version), but it\'s completely respectable and pleasingly different in parts.<br /><br />I am an English teacher and I find some of the ignorance about language in some of these reviews rather upsetting. For example: the "name should scream don\'t watch. \'How she move.\' Since when can movie titles ignore grammar?" <br /><br />There is nothing inherently incorrect about Caribbean English grammar. It\'s just not Canadian standard English grammar. Comments about the dialogue seem off to me. I put on the subtitles because I\'m a Canadian standard English speaker, so I just AUTOMATICALLY assumed that I would have trouble understanding all of it. It wasn\'t all that difficult and it gave a distinctly different flavour as the other step movies I have seen were so American.<br /><br />I loved that this movie was set in Toronto and, in fact, wish it was even more c

Perfect! As expected, we see that the pos_fl0 contains a single item and it's value output matches the output of the previous code cell. 

### Preprocessing the reviews

Having, learnt how to use tf.data.TextLineData() method, we can now starting preprocessing the data. In order to do this, we notice that the review contains punctuation marks and html line break tags etc. We will have to write a preprocessing function to get rid of these. Additionally, we will also change all alphabets to lower case.

#### Removing line-brk tags

This can be very simply done by using the .replace() method of python strings. We can therefore use it to replace all occurrances of the line-break tag with a space. In tensorflow, the equivalent method is [tf.strings.regex_replace()](https://www.tensorflow.org/api_docs/python/tf/strings/regex_replace).

#### tf.strings.regex_replace
Note that 'regex' in regex_replace() stands for ["regular expression"](https://docs.python.org/3/howto/regex.html). For e.g. the following will work:

In [14]:
tf.strings.regex_replace('hello','e','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

But the following will throw an error: tf.strings.regex_replace('h(llo', '(','E')

This is because "(" is a metacharacter. To match and replace meta-characters, we must prepend a backslash before them. This can be done as follows: '\\\\' + char. 

The previous code cell can now be made to work

In [15]:
# The error thrown by this code cell is intentional
tf.strings.regex_replace('h(llo', '\\'+'(','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

or equivalently:

In [16]:
tf.strings.regex_replace('h(llo','\(','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

##### Imp: 

Note that backslash itself is also a  meta-character. To search and replace backslash, we do the following:

In [17]:
tf.strings.regex_replace('h\llo', '\\'+'\\', 'E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

or equivalently:

In [18]:
# we use '\\\\' and NOT '\\' to search and replace a backslash
tf.strings.regex_replace('h\llo', '\\\\', 'E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

We now try three different ways of removing punctuations from a tensorflow string and compare their timings:

1) punc_filter_and_to_lower1: use tf.strings.unicode_decode() to convert all the characters in the string into an array of their ascii codes. We then loop through this array, skipping over the places where the entry matches the ascii code of a punctuation. Finally we call tf.strings.unicode_encode() on this array to convert the ascii codes back to characters, thereby obtaining a string with all the punctuations stripped. 


2) punc_filter_and_to_lower2: use tf.regex_replace() to search and replace each punctuation by empty space. Pay special attention to prepend backslash in order to be able to use meta-characters in regex_replace.


3) punc_filter_and_to_lower3: simply extract the python string using its .numpy() method. Then simple iterate through the characters of the string, skipping over the punctutations. Join the resulting list of character using .join() method. 

4) punc_filter_and_to_lower4: Use [tfds.features.text.Tokenizer()](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) by setting it's "alphanum_only" arg to True. This way it will only parse alpha-numeric characters in the text and to split the text at occurrances of non-alphanumeric characters. Since whitespace is a non-alpha-numeric character, the output will largely consist of a list of words in the text.  The caeat with this approach is that words with apostrophe in them such as " don't " will be split into two words: "don" and "t". Also, [tfds.features.text.Tokenizer()](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) does NOT consider underscores as non-alpha-numeric, so underscores do not get removed form the text. On the other hand since, the tokenizer already generates a list of words, we use this list to generate a vocobulary of words in the dataset at this step it-self, making the preprocessing faster.

There timing on the first review in the training set was as follows:

1) punc_filter_and_to_lower1: Wall time: ~ 3 s

2) punc_filter_and_to_lower2: Wall time: ~ 8 ms

3) punc_filter_and_to_lower3: Wall time: ~ 6 ms

4) punc_filter_and_to_lower4: Wall time: ~ 6 ms (when not udating a vocabulary of words) 

5) 4) punc_filter_and_to_lower4: Wall time: ~ 7 ms (when also udating a vocabulary of words) 

Clearly, the last function is the fastest with the first one being extremely slow (takes several secs). 

In [19]:
import string

In [20]:
# list of punctuations
punc_ls = string.punctuation
print('punctuations before utf encoding: {}'.format(punc_ls))
punc_ls2 = tf.strings.unicode_decode(punc_ls, input_encoding = 'UTF-8')
print('punctuations after utf encoding: \n{}'.format(punc_ls2))

punctuations before utf encoding: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
punctuations after utf encoding: 
[ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  58  59  60
  61  62  63  64  91  92  93  94  95  96 123 124 125 126]


In [21]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tf.strings.unicode_encode(), tf.strings.unicode_decode()
def punc_filter_and_to_lower1(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ') # regex stands for regular expressions
                                                          # i.e. they are not metacharacters
                                                          # https://docs.python.org/3/howto/regex.html#matching-characters
    
    # now we replace all the punctions in the string
    st2 = tf.strings.unicode_decode(st2, input_encoding = 'utf-8')
    st2 = tf.strings.unicode_encode([char for char in st2 if char not in punc_ls2], output_encoding = 'UTF-8')
    
    st2 = tf.strings.lower(st2)
    
    return st2

In [22]:
import time

In [23]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower1(item)

CPU times: user 3.43 s, sys: 60.8 ms, total: 3.49 s
Wall time: 3.49 s


In [24]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all its not better than the amazing ones strictly ballroom shall we dance japanese version but its completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream dont watch how she move since when can movie titles ignore grammar  there is nothing inherently incorrect about caribbean english grammar its just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because im a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasnt all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was so atypic

In [25]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tf.strings.regex_replace()
def punc_filter_and_to_lower2(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ')
    
    # now we replace all the punctions in the string
    for punc in punc_ls:
        # V.imp: to replace meta-characters we prepend a backslash to them 
        # infact we can also prepend a backslach before all the characters
        # to prepend a backslash before a character we do: '\\' + char
        st2 = tf.strings.regex_replace(st2, '\\'+ punc, ' ')
    
    st2 = tf.strings.lower(st2)
    
    return st2

In [26]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower2(item)

CPU times: user 8.48 ms, sys: 90 µs, total: 8.57 ms
Wall time: 7.91 ms


In [27]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all  it s not better than the amazing ones   strictly ballroom    shall we dance    japanese version   but it s completely respectable and pleasingly different in parts  i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting  for example  the  name should scream don t watch   how she move   since when can movie titles ignore grammar    there is nothing inherently incorrect about caribbean english grammar  it s just not canadian standard english grammar  comments about the dialogue seem off to me  i put on the subtitles because i m a canadian standard english speaker  so i just automatically assumed that i would have trouble understanding all of it  it wasn t all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american  i loved that this movie was set in toronto and  in fact  wish it was even more clearly set there  i 

In [28]:
# function to get rid of punctuations, html line breaks and change to lower case
# using .join() method in python string class
def punc_filter_and_to_lower3(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ') # regex stands for regular expressions
                                                          # i.e. they are not metacharacters
                                                          # https://docs.python.org/3/howto/regex.html#matching-characters
    
    # now we replace all the punctions in the string
    st2 = st2.numpy().decode('utf-8')
    st2 = ''.join([char for char in st2 if char not in punc_ls])
    
    st2 = st2.lower()
    
    st2 = tf.convert_to_tensor(st2)
    
    return st2

In [29]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower3(item)

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 5.02 ms


In [30]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all its not better than the amazing ones strictly ballroom shall we dance japanese version but its completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream dont watch how she move since when can movie titles ignore grammar  there is nothing inherently incorrect about caribbean english grammar its just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because im a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasnt all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was so atypic

In [31]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tfds.features.text.tokenizer()
# we will also simultaneously generate a vocabulary in this step
import tensorflow_datasets as tfds
def punc_filter_and_to_lower4(st, vocab = None):
    ''' vocab: vocabulary to update with the words in st'''
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.lower(tf.strings.regex_replace(st, line_brk_tag, ' '))
    
    # now we replace all the punctions in the string
    tokenizer = tfds.features.text.Tokenizer()
    words = tokenizer.tokenize(st2.numpy()) # note that the inpout has to be a python string NOT as tensor 
                                            # The output is a list NOT a tensor
                                            # https://stackoverflow.com/questions/56665868/tensor-numpy-not-working-in-tensorflow-data-dataset-throws-the-error-attribu
    st2 = tf.strings.join(words, ' ')
    
    if type(vocab) == set:
        vocab.update(words)
        return st2, vocab
    
    
    return st2

In [32]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower4(item)

CPU times: user 8.14 ms, sys: 22 µs, total: 8.16 ms
Wall time: 7.44 ms


In [33]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all it s not better than the amazing ones _strictly ballroom_ _shall we dance _ japanese version but it s completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream don t watch how she move since when can movie titles ignore grammar there is nothing inherently incorrect about caribbean english grammar it s just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because i m a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasn t all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was

In [34]:
%%time
# test the timing of punc_filter_and_to_lower4 when simultaneously updating a dictionary
vocab2 = set([])
for item in pos_fl0:
    lst, vocab2 = punc_filter_and_to_lower4(item, vocab2)

CPU times: user 8.46 ms, sys: 79 µs, total: 8.53 ms
Wall time: 7.93 ms


### Build a Vocabulary based on the training data

This can be easily done using python's [set()](https://docs.python.org/3.8/library/stdtypes.html#set-types-set-frozenset) container.: basically, split each text in the training instance into words and update the set with this list. This will add any new words in the text to the set. In the end, the set will contain all the unique words in the training dataset.

Note that we have also implemented the above idea in our function punc_filter_and_to_lower4() 

In [35]:
# function to create a vocabulary using the set() container
vocab1 = set([]) # python set for containing unique words in the training dataset
def vocab_builder1(strng):
    # split the string into its words
    words = tf.strings.split(strng)
    # update vocab
    vocab1.update(words.numpy())
    return

In [36]:
%%time
for item in pos_fl0:
    item = punc_filter_and_to_lower3(item)
    vocab_builder1(item)

CPU times: user 262 ms, sys: 64 ms, total: 326 ms
Wall time: 324 ms


Note that the vocab_builder1() function as defined above takes about 300 ms. On the otherhand, in punc_filter_and_to_lower4() function,  we were able to get almost the same result (upto the caveats mentioned in the previous section) by using tfds.features.text.Tokenizer(), in about 8 ms. Clearly, there is a difference of few order of magnitude between the time taken by the two routines with the difference between their output being quite tolerable.  

#### building the vocabulary

In [37]:
vocab = set([])
train_filepaths = tf.data.Dataset.list_files([train_pos_path, train_neg_path])
train_dataset = tf.data.TextLineDataset(train_filepaths)

ctr = 0
for item in train_dataset:
    _ , vocab = punc_filter_and_to_lower4(item, vocab)
    
    ctr+=1
    if ctr%255 ==0:
        print("=", end = '')

In [38]:
# pick 5 random words from vocab to inspect it
import random

random.sample(vocab, 5)

['bloodletting', 'cardella', 'degree', 'utilize', 'taos']

In [39]:
print('no. of words in vocabulary: {}'.format(len(vocab)))

no. of words in vocabulary: 74893


In [40]:
word_indices = { item:index for index, item in enumerate(vocab)}
len(word_indices)

74893

### Labeling the data

Note that the positive and negative reviews in each dataset have been stored in seperate folders and as such they have no explicit labels. We will therefore have to create our own labels as was done in [this](https://www.tensorflow.org/tutorials/load_data/text) official tensorflow tutorial. 

In [41]:
# function to return apply a label
def labeler(review, label):
    return review, tf.cast(label, tf.int64)

In [42]:
train_pos_paths = tf.data.Dataset.list_files(train_pos_path)
train_pos_reviews = tf.data.TextLineDataset(train_pos_paths).map(lambda x : labeler(x, 1))

In [43]:
for item in train_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'Picked up the movie at the flea market for 4 bucks, sure did get my moneys worth!. Could care-less about the hot babes but the animation just blew me away after a steady diet of Simpsons (Sorry Mr. Groening). The best part, facial expressions. Recommend multiple viewings with some cool tunes, good friends and a couple of cold ones!'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"Good for an evening's entertainment - but the plot was unconvincing. Garrison's affair with the First Lady was unreal and passionless; the President was a cardboard cut-out. And who were the real villains anyway? Nothing was developed or explained sufficiently. I still don't know why they wanted to kill the President or how the mole got involved. The villains were nameless and undeveloped, so you never felt involved in their plot. Michael Douglas and Kiefer Sutherland did their best to inject some reality into the story 

### Creating training batches

In [44]:
train_neg_paths = tf.data.Dataset.list_files(train_neg_path)
train_neg_reviews = tf.data.TextLineDataset(train_neg_paths).map(lambda x: labeler(x, 0))

In [45]:
for item in train_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'Having just recently re-viewed "Lipstick" for the first time in a few decades, I backed it with "Descent" even though I have heard more negative comments than good from other film friends with tastes as varied as mine.<br /><br />It\'s interesting to contrast how the unique niche of the Rape Revenge movie has evolved in the past 32 years, from the full-on gore of "I Spit On Your Grave," to the tawdry sensationalism of "Lipstick," to the tasteful handling of the issue in "The Accused." But "Descent," though making some important points, never really offers us anything truly new in terms of revelatory meaning. No, "Descent" is so poorly made in terms of picture and sound quality that it detracts from any significant message it could hope to make --- a message that, when examined closely, isn\'t that groundbreaking.<br /><br />I pretty much knew the plot going in. What I wanted to see *was* the "descent" or degeneration of Dawson\'s character. 

In [46]:
shuffle_buffer_size = 20000
train_dataset = train_pos_reviews.concatenate(train_neg_reviews).shuffle(buffer_size = shuffle_buffer_size)

In [47]:
count = 0
for item in train_dataset:
    count+=1
print(count)    

25000


We now wish to apply punc_filter_and_to_lower4() to the reviews in our training dataset. For this we will use the [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) method of tf.data.Datasets. At this point it is important to note that (as mentioned in the documentation for [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map)):

    Note that irrespective of the context in which map_func is defined (eager vs. graph), tf.data traces the function and executes it as a graph. To use Python code inside of the function you have two options:

     1) Rely on AutoGraph to convert Python code into an equivalent graph computation. The downside of this approach is that AutoGraph can convert some but not all Python code.

     2) Use tf.py_function, which allows you to write arbitrary Python code but will generally result in worse performance than 1)

This point is of concern to us because the [tokenizer](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) we used in punc_filter_and_to_lower4() only accepts string input and NOT tensors. This string was extracted from the tensor using its .numpy() method which is only operable in eager mode. Thus when we pass punc_filter_and_to_lower4() to map(), it throws an error, complaining:

     'Tensor' object has no attribute 'numpy'


Thus we have to wrap punc_filter_and_to_lower4() with tf.py_function before passing it to map


In [48]:
# function to wrap punc_filter_and_to_lower4 with tf.py_function
# we will also use tf.ensure_shape(), or else tensorflow is 
# unable to statically determine it, leading to error being thrown during training
def string_transform(X):
    x = tf.py_function(punc_filter_and_to_lower4, [X], Tout = tf.string)
    x = tf.ensure_shape(x, ())
    
    return x

batch_size = 50
prefetch = 2
train_batch = train_dataset.map(lambda X, y: 
                                    (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

In [88]:
ite = next(iter(train_batch))
print('shape of train-batch: {}'.format(ite[0].shape))
print(ite[0][0])

shape of train-batch: (50,)
tf.Tensor(b'in 1976 i had just moved to the us from ceylon i was 23 and had been married for a little over three years and was beginning to come out as a lesbian i saw this movie on an old black and white tv with terrible reception alone and uninterrupted in an awakening that seemed like an echo of the story i was living in a small house in tucson arizona and it was summertime like everyone else here i never forgot the feelings the images of this story called forth and its residue of fragile magic and i have treasured a hope that i would see it again someday i ll keep checking in i also wish that someone would make a movie of shirley verel s the other side of venus it also has some of the same delicacy and persistent poignancy', shape=(), dtype=string)


In [93]:
ite[1]

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])>

### Create Validation dataset 

As instructed in the excercise, we will create a validation dataset by splitting the test set into half

In [55]:
test_path = os.path.join(path, 'test')
print(test_path)
test_pos_path = os.path.join(test_path, 'pos', '*.txt')
print('path to positive test reviews: \n{}'.format(test_pos_path))
test_neg_path = os.path.join(test_path, 'neg', '*.txt')
print('path to negative test reviews: \n{}'.format(test_neg_path))

/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test
path to positive test reviews: 
/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test/pos/*.txt
path to negative test reviews: 
/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test/neg/*.txt


In [68]:
test_pos_files = glob.glob(test_pos_path)
len(test_pos_files)

12500

In [69]:
valid_size = 12500
valid_pos_files = random.sample(test_pos_paths, int(valid_size/2))
len(valid_pos_files)

6250

In [70]:
# We can remove the valid_pos_files from the set of test_pos_files through the
# simple trick explained in the following post:
# https://stackoverflow.com/questions/6486450/python-compute-list-difference/6486467
test_pos_files = list(set(test_pos_files) - set(valid_pos_files))
len(test_pos_files)

6250

In [76]:
test_neg_files = glob.glob(test_neg_path)
print('initial test_neg_files len:{}'.format(len(test_neg_files)))
valid_neg_files = random.sample(test_neg_files, int(valid_size/2))
print('valid_neg_files len: {}'.format(len(valid_neg_files)))
test_neg_files = list(set(test_neg_files) - set(valid_neg_files))
print('test_neg_files len: {}'.format(len(test_neg_files)))

initial test_neg_files len:12500
valid_neg_files len: 6250
test_neg_files len: 6250


In [78]:
valid_pos_reviews = tf.data.TextLineDataset(valid_pos_files).map(lambda x: labeler(x, 1))
valid_neg_reviews = tf.data.TextLineDataset(valid_neg_files).map(lambda x: labeler(x, 0))

In [79]:
for item in valid_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'After watching John preform this one of a kind show, I had to share.....It was really something to watch a grown man portray himself as a child. I like the fact that with every character he "became," you could picture what they looked like. It is more entertaining when you can understand the individual. "Freak" is what real "stand up" should be. John is REAL talent.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"I like Dylan Moran from his work in Black Books, although I found some of his stand-up to be really indulgent in terms of long confused gaps... however I was intrigued to see this film starring he and Michael Caine and curiosity got the better of me.<br /><br />I was stunned.<br /><br />Dylan's vocal range and characterisation of the different people he was playing in the film was absolutely perfect, something beyond the skills of a mere stand-up comedian and really truly on a par with 

In [80]:
for item in valid_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"I usually give sequels the benefit of the doubt and go easy on them. But this\xc2\x85is very poor. The exact same as what happened in the first film happens here again. The exact same. Only they wanted younger kids to be able see it (this accounts for the repeated presence of Bugs Bunny but why they threw in one 'f*ck' is beyond me).<br /><br />They didn't even bother to change Jackie Mason's character from Dangerfield's. Let me explain, in CADDYSHACK Rodney Dangerfield played a boorish Real Estate owner who enrages the uptight members of Bushwood country club. In this movie Jackie Mason plays a boorish Real Estate owner who enrages the uptight members of Bushwood country club. A big effort they made to change the content of the script huh? No, I don't think so. A very unfunny Dan Aykroyd shamefully copies Bill Murray's character. He even goes after the gopher (now more like a Gremlin) in much the same fashion as Murray did. <br /><br />Ther

In [82]:
test_pos_reviews = tf.data.TextLineDataset(test_pos_files).map(lambda x: labeler(x, 1))
test_neg_reviews = tf.data.TextLineDataset(test_neg_files).map(lambda x: labeler(x, 0))

In [83]:
for item in test_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'The only pure life, is one that ends with a signature in blood.<br /><br />So says Mishima anyway, a young sheltered boy who becomes a celebrity author. The life of one of Japans most celebrated literary voices, is told from three perspectives, his life just before he and four members of his private army take over a Japanese military base and commit ritual suicide(shown in color), flashbacks(shown in black and white), and scenes from his novels(shown in a kind of dreamy Technicolor set design somewhere between traditional Noh Theater and "the Wizard Of Oz". These stories are often told at the same time, but are edited to reinforce, the slow fusing of Mishima\'s life with his fictions, until the end(or the beginning) when like the ancient samurai he so admires, he will be at a balance of pen and sword (when his words and actions are the same, and he is a full and "pure" being).<br /><br />Paul Schrader wrote the screen play for "Taxi Driver",

In [85]:
for item in test_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'Lazy movie made by a lazy director. The characters are grotesque. Despite the tragic of this war, there is no emotion at all in the movie. Symbolism is artificial and inefficient (and old Bosnian woman giving a photo of her son to Arbour will "concretize" her willingness, will awake the super-mother sleeping inside her, a corpse eaten by worms to show the horror of genocide... too much is sometimes worst than not enough).<br /><br />This movie is only an advertisement, an empty elegy to a woman who is not a hero. She worked for United Nations. Remember UN failed to protect civilians at Srebrenica. Who are the true heroes of this war? A Canadian judge leading post-mortem trial for atrocities that happened mostly because her organization failed to prevent them? Where is the criticism in this movie?'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"I love exotic science fiction/fantasy movies but thi

In [86]:
buffer_size = 10000
valid_dataset = valid_pos_reviews.concatenate(valid_neg_reviews).shuffle(buffer_size = buffer_size)

In [89]:
valid_batch = valid_dataset.map(lambda X,y: 
                                (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

In [91]:
test_dataset = test_pos_reviews.concatenate(test_neg_reviews).shuffle(buffer_size)

In [92]:
test_batch = test_dataset.map(lambda X,y: 
                                (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)